<a href="https://colab.research.google.com/github/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/5-generating-text-with-rnn-and-gpt-2/02_generating_text_with_greedy_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Generating text – Overview

When your mobile phone completes a word as you type a message or when Gmail
suggests a short reply or completes a sentence as you reply to an email, a text
generation model is working in the background.

The Transformer architecture forms the basis of state-of-the-art text generation models. **BERT uses only the encoder part of the Transformer architecture.**

However, BERT, being bi-directional, is not suitable for the generation of text. A left-to-right (or right-to-left, depending on the language) language model built on the decoder part of the Transformer architecture is the foundation of text generation models today.

Text can be generated a character at a time or with words and sentences together.

Specifically, we will cover the following topics:

- Generating text with:
  - Character-based RNNs for generating news headlines and completing text messages
  - GPT-2 to generate full sentences

- Improving the quality of text generation using techniques such as:
  - Greedy search
  - Beam search
  - Top-K sampling

- Using advanced techniques such as learning rate annealing and
checkpointing to enable long training times
- Details of the Transformer decoder architecture
- Details of the GPT and GPT-2 models

A character-based approach for generating text is shown first. Such models can be quite useful for generating completions of a partially typed word in a sentence on a messaging platform, for example.



##Setup

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
import os
import datetime
import time
import matplotlib.pyplot as plt

tf.__version__

'2.6.0'

In [2]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

tf.keras.backend.clear_session() #- for easy reset of notebook state

# chck if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


In [3]:
%%capture

!wget https://github.com/PacktPublishing/Advanced-Natural-Language-Processing-with-TensorFlow-2/raw/master/chapter5-nlg-with-transformer-gpt/char-rnn/news-headlines.tsv

##Generating text with greedy search

There are two main steps in generating text. The first step is restoring a trained model from the checkpoint. The second step is generating a character at a time from a trained model until a specific end condition is met.

##Data loading and pre-processing

For this particular example, we are going to use data from a constrained domain – a
set of news headlines. The hypothesis is that news headlines are usually short and
follow a particular structure. These headlines are usually a summary of an article and
contain a large number of proper nouns like names of companies and celebrities.

The first dataset is called the [News Aggregator dataset](https://archive.ics.uci.edu/ml/datasets/News+Aggregator).
This dataset has
over 420,000 news article titles, URLs, and other information.

The second dataset is a
set of over 200,000 news articles from The Huffington Post, called the [News Category dataset](https://www.kaggle.com/rmisra/news-category-dataset).

News article headlines from both datasets are extracted and compiled into one file.

Let's inspect the contents of the file to get a sense of the data:
$ head






In [4]:
!head -5 news-headlines.tsv

There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV	there were 2 mass shootings in texas last week, but only 1 on tv
Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song	will smith joins diplo and nicky jam for the 2018 world cup's official song
Hugh Grant Marries For The First Time At Age 57	hugh grant marries for the first time at age 57
Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork	jim carrey blasts 'castrato' adam schiff and democrats in new artwork
Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog	julianna margulies uses donald trump poop bags to pick up after her dog


## Data normalization and tokenization

This model uses a token per character. So, each letter, including
punctuation, numbers, and space, becomes a token. Three additional tokens are
added. These are:

- `<EOS>`: Denotes end of sentences
- `<UNK>`: it is common to replace out-of-vocabulary words with a special token
- `<PAD>`: This is a unique padding token used to pad all headlines to the
same length

To start, the tokenization function needs to be set up:

In [5]:
chars = sorted(set("abcdefghijklmnopqrstuvwxyz0123456789 -,;.!?:’’’/\|_@#$%ˆ&*˜‘+-=()[]{}' ABCDEFGHIJKLMNOPQRSTUVWXYZ"))
chars = list(chars)

EOS = "<EOS>"
UNK = "<UNK>"
PAD = "<PAD>"  # need to move mask to '0'index for Embedding layer

chars.append(UNK)
chars.append(EOS)  # end of sentence

chars.insert(0, PAD)  # now padding should get index of 0

Once the token list is ready, methods need to be defined for converting characters to tokens and vice versa. 

Creating mapping is relatively straightforward:

In [6]:
# Creating a mapping from unique characters to indices
char2index = {u: i for i, u in enumerate(chars)}
index2char = np.array(chars)

def char_index(ch):
  # takes a character and returns an index
  # if character is not in list, returns the unknown token
  if ch in chars:
    return char2index[ch]
  return char2index[UNK]

A maximum length of 75
characters is used for the headlines. If the headlines are shorter than this length,
they are padded. Any headlines longer than 75 characters are snipped. The <EOS>
token is appended to the end of every headline.

Let's set this up:

In [7]:
data = []     # load into this list of lists
MAX_LEN = 75  # maximum length of a headline

with open("news-headlines.tsv", "r") as f:
  lines = csv.reader(f, delimiter="\t")
  for line in lines:
    headline = line[0]
    converted = [char_index(c) for c in headline[:-1]]  # convert to number

    if len(converted) >= MAX_LEN:
      converted = converted[0: MAX_LEN - 1]
      converted.append(char2index[EOS])
    else:
      converted.append(char2index[EOS])
      # add padding tokens
      remaining = MAX_LEN - len(converted)
      if remaining > 0:
        for i in range(remaining):
          converted.append(char2index[PAD])
    data.append(converted)
print("**** Data file loaded ****")

**** Data file loaded ****


You may be wondering about
the ground truth here for training as we only have a line of text. Since we want this
model to generate text, the objective can be reduced to predicting the next character
given a set of characters. 

Hence, a trick will be used to construct the ground truth
– we will just shift the input sequence by one character and set it as the expected
output.

This transformation is quite easy do with numpy:

In [8]:
# now convert to numpy array
np_data = np.array(data)

# for training, we use one character shifted data
np_data_in = np_data[:, :-1]
np_data_out = np_data[:, 1:]

With this nifty trick, we have both inputs and expected outputs ready for training.

The final step is to convert it into tf.Data.DataSet for ease of batching and shuffling:

In [9]:
# Create TF dataset
X = tf.data.Dataset.from_tensor_slices((np_data_in, np_data_out))

Now everything is ready to start training.

## Training the model

Since the checkpoints only stored the weights for the layers, defining the model structure is important. The main difference from the training network is the batch size. We want to generate a sentence at a time, so we set the batch size as 1:

In [10]:
# Length of the vocabulary in chars
vocab_size = len(chars)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024
# batch size
BATCH_SIZE = 256

A convenience function for setting up the model structure is defined like so:

In [11]:
#this one is without padding masking or dropout layer
def build_gen_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer="glorot_uniform"),
      tf.keras.layers.Dense(vocab_size)                         
  ])

  return model

A model can be instantiated with this method:

In [12]:
gen_model = build_gen_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

Note that the embedding layer does not use masking because, in text generation,
we are not passing an entire sequence but only part of a sequence that needs to be completed. 

Now that the model is defined, the weights for the layers can be loaded
in from the checkpoint. Please remember to replace the checkpoint directory with
your local directory containing the checkpoints from training:

In [14]:
# Now setup the location of the checkpoint and load the latest checkpoint
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints/2020-Oct-01-14-29-55"

gen_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
gen_model.build(tf.TensorShape([1, None]))

print("**** Model Instantiated ****")
print(gen_model.summary())

AttributeError: ignored

The second main step is to generate text a character at a time. Generating text needs a seed or a starting few letters, which are completed by the model into a sentence.

The process of generation is encapsulated in the function below:

In [ ]:
# Low temperatures results in more predictable text.
# Higher temperatures results in more surprising text.
# Experiment to find the best setting.
def generate_text(model, start_string, temperature=0.7, num_generate=75):
  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx(s) for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Here batch size == 1
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the word returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

    # We pass the predicted word as the next input to the model along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

    # lets break is <EOS> token is generated
    if idx2char[predicted_id] == EOS:
      break #end of a sentence reached, lets stop
  
  return (start_string + "".join(text_generated))

At every point, the character with the highest likelihood is chosen. Choosing the
next letter with the highest probability is called greedy search. However, there
is a configuration parameter called temperature, which can be used to adjust the
predictability of the generated text.

Once probabilities for all characters are predicted, dividing the probabilities by the temperature changes the distribution of the generated characters. Smaller values of the temperature generate text that is closer to the original text. Larger values of the temperature generate more creative text. Here, a value of 0.7 is chosen to bias more on the surprising side.

To generate the text, all that is needed is one line of code:



In [ ]:
print(generate_text(gen_model, start_string=u"Google"))

In [ ]:
print(generate_text(gen_model, start_string=u"Apple"))

Each execution of the command may generate slightly different results. The line
generated above, while obviously nonsensical, is pretty well structured. The model has learned capitalization rules and headline structure.

##Implementing learning rate decay

There are two ways to implement learning rate decay in TensorFlow. The first way
is to use one of the prebuilt schedulers that are part of the
`tf.keras.optimizers.schedulers` package and use a configured instance with the optimizer.

An example of a prebuilt scheduler is InverseTimeDecay.

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,          # initial learning rate
    decay_steps=STEPS_PER_EPOCH*(EPOCHS/10),
    decay_rate=2,
    staircase=False
)

The
number of steps per epoch can be calculated by dividing the number of training
examples by batch size. The number of decay steps determines how the learning
rate is reduced. The equation used to compute the learning rate is:

$$
new\_rate = \frac{initial\_rate}{1 + decay\_rate * (\frac{step}{decay\_step})} 
$$

After being set up, all this function needs is the step number for computing the new learning rate. Once the schedule is set up, it can be passed to the optimizer:

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr_schedule)

That's it! The rest of the training loop code is unchanged. 

However, this learning rate scheduler starts reducing the learning rate from the first epoch itself. A lower learning rate increases the amount of training time. Ideally, we would keep the learning rate unchanged for the first few epochs and then reduce it.

Looking at plot above, the learning rate is probably effective until about the
tenth epoch.

BERT also uses learning rate warmup before learning rate decay.
Learning rate warmup generally refers to increasing the learning rate for a few
epochs. BERT was trained for 1,000,000 steps, which roughly translates to 40 epochs.
For the first 10,000 steps, the learning rate was increased, and then it was linearly
decayed. Implementing such a learning rate schedule is better accomplished by a
custom callback.

Custom callbacks in TensorFlow enable the execution of custom logic at various
points during training and inference. We saw an example of a prebuilt callback that saves checkpoints during training. 

A custom callback provides hooks that enable
desired logic that can be executed at various points during training. This main
step is to define a subclass of `tf.keras.callbacks.Callback`. Then, one or more of the following functions can be implemented to hook onto the events exposed by TensorFlow:

- `on_[train,test,predict]_begin / on_[train,test,predict]_end`: This
callback happens at the start of training or the end of the training.

- `on_[train,test,predict]_batch_begin / on_[train,test,predict] _batch_
end`: These callbacks happen when training for a specific batch starts or ends.

- `on_epoch_begin / on_epoch_end`: This is a training-specific function called at the start or end of an epoch.

We will implement a callback for the start of the epoch that adjusts that epoch's
learning rate. Our implementation will keep the learning rate constant for a
configurable number of initial epochs and then reduce the learning rate in a fashion
similar to the inverse time decay function.

<img src='https://github.com/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/5-generating-text-with-rnn-and-gpt-2/images/1.png?raw=1' width='800'/>

First, a subclass is created with the function defined in it. The best place to put this, is just around the checkpoint callback, before the start of training.

In [ ]:
# Custom Callback for Learning Rate Decay
class LearningRateScheduler(tf.keras.callbacks.Callback):
  """Learning rate scheduler which decays the learning rate"""

  def __init__(self, init_lr, decay, steps, start_epoch):
    super().__init__()
    self.init_learning_rate = init_lr   #initial learning rate
    self.decay = decay                  # how sharply to decay
    self.steps = steps                  # total number of steps of decay
    self.start_epoch = start_epoch      # which epoch to start decaying

  def on_epoch_begin(self, epoch, logs=None):

    if not hasattr(self.model.optimizer, "lr"):
      raise ValueError('Optimizer must have a "lr" attribute.')
    # Get the current learning rate from model's optimizer.
    lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))

    if (epoch >= self.start_epoch):
      # Call schedule function to get the scheduled learning rate.
      scheduled_lr = self.init_learning_rate / (1 + self.decay * (epoch / self.steps))
      # Set the value back to the optimizer before this epoch starts
      tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
      print("\nEpoch %05d: Learning rate is %6.8f." % (epoch, scheduled_lr))

Using this callback in the training loop requires the instantiation of the callback. The following parameters are set while instantiating the callback:

- The initial learning rate is set to 0.001.
- The decay rate is set to 4. Please feel free to play around with different
settings.
- The number of steps is set to the number of epochs. The model is trained for
150 epochs.
- Learning rate decay should start after epoch 10, so the start epoch is set to 10.

The training loop is updated to include the callback like so:

In [ ]:
print("**** Start Training ****")

EPOCHS = 150
lr_decay = LearningRateScheduler(0.001, 4., EPOCHS, 10)

start = time.time()

history = model.fit(x_train, epochs=EPOCHS, callbacks=[checkpoint_callback, lr_decay])

print("**** End Training ****")
print("Training time: ", time.time()- start)

print("Checkpoint directory: ", checkpoint_dir)

Now, the model is ready to be trained. Training 150 epochs took over 10 hours on the GPU-capable machine.


In [ ]:
# Plot accuracies
lossplot = "loss-" + dt + ".png"
plt.plot(history.history['loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

The loss drops very fast for the first few epochs before plateauing
near epoch 10. Learning rate decay kicks in at that point, and the loss starts to fall again.

Training this model took much time and advanced tricks like learning rate decay to train.